<a href="https://colab.research.google.com/github/lmmlima/ENV716_EnergyModeling_F2021/blob/main/Lab11/Lab11_A7_SOLUTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 11 - A7 Solution**


Learning goals for Lab11:
* More on abstract models;
* Solve A7.

## Initializing 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

Installing Pyomo and solver. Recall for teh shortest path example we have binary variables so we will need to use another solver. Instead of installing glpk, thsi time we will install COIN-OR CBC. \\ 
COIN-OR CBC is a multi-threaded open-source Coin-or branch and cut **mixed-integer linear programming solver**. CBC is generally a good choice for a general purpose MILP solver for medium to large scale problems.

In [ ]:
!pip install pyomo
#!apt-get install -y -qq glpk-utils
!apt-get install -y -qq coinor-cbc

     |████████████████████████████████| 9.1 MB 4.9 MB/s 
     |████████████████████████████████| 49 kB 6.3 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155219 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unp

Importing pyomo and cbc solver.

In [ ]:
from pyomo.environ import *
#Import solver
opt=SolverFactory('cbc',executable='/usr/bin/cbc')

## Q1: Maximum Flow Model - Natural Gas company

Let's start by writing the max flow problem.

**Sets** \\
$N$: set of nodes \\
$K$: set of nodes without source and sink nodes \\
$A$: set of arcs $(ij)$ \\

**Parameters** \\
$u_{ij}$: flow capacity for arc $(ij)$

**Decision Variable** \\
$x_{ij}$: how much flow on arc $(ij)$ - any value from 0 to $u_{ij}$

**Model** \\
$ min \ \sum_{j \in N} x_{1j}$ \\
$ s.t.$
$ \ \sum_{j \in N} x_{ij} = \sum_{j \in N} x_{ji} \quad \forall i \in K $ \\
$ \quad \quad \sum_{j \in N} x_{1j} = \sum_{j \in N} x_{j10} $ \\
$ \quad \quad x_{ij} \leq u_{ij} \quad \forall (ij) \in A $ \\
$ \quad \quad x_{ij} \geq 0 \quad \forall (ij) \in A $


In [ ]:
model=ConcreteModel()

model.Nodes=Set(initialize=range(1,11))  
model.NodesK=Set(within=model.Nodes,initialize=range(2,10))

model.first=1
model.last=10

model.Arcs=Set(within=model.Nodes*model.Nodes, 
                initialize=[(1,2),(1,3),(1,4),(2,5),(5,2),
                            (3,4),(3,5),(3,6),(5,3),(6,3),
                            (4,6),(4,10),(6,4),(5,7),(5,9),(9,5),
                            (6,7),(6,8),(6,10),(7,6),(8,6),
                            (7,8),(7,9),(8,7),(9,7),(8,9),(8,10),(9,8),(9,10)])

model.FlowCap=Param(model.Arcs,
                     initialize={(1,2):5,(1,3):12,(1,4):8,(2,5):6,(5,2):3,
                            (3,4):2,(3,5):4,(3,6):5,(5,3):3,(6,3):4,
                            (4,6):9,(4,10):2,(6,4):5,(5,7):6,(5,9):5,(9,5):2,
                            (6,7):3,(6,8):6,(6,10):8,(7,6):4,(8,6):3,
                            (7,8):5,(7,9):7,(8,7):2,(9,7):3,(8,9):5,(8,10):7,(9,8):1,(9,10):4})

#Add dec variables
model.x=Var(model.Arcs,domain=NonNegativeReals)

#Adding objective function
def max_flow(model2):
    return sum(model.x[i,j] for (i,j) in model2.Arcs if i == model.first)
model.maxflow=Objective(rule=max_flow, sense=maximize)

print(model.maxflow.expr)


x[1,2] + x[1,3] + x[1,4]


In [ ]:
#Adding constraints
def flow_bal(model, n):
    inFlow  = sum(model.x[i,j] for (i,j) in model.Arcs if j == n)
    outFlow = sum(model.x[j,i] for (j,i) in model.Arcs if j == n)
    return inFlow == outFlow
model.flowbalance = Constraint(model.NodesK, rule=flow_bal)

for n in model.NodesK:
  print(model.flowbalance[n].expr)

def orig_dest(model):
    orig  = sum(model.x[i,j] for (i,j) in model.Arcs if i == model.first)
    dest = sum(model.x[i,j] for (i,j) in model.Arcs if j == model.last)
    return orig == dest
model.origdest = Constraint(rule=orig_dest)

print(model.origdest.expr)

def flow_cap(model,i,j):
    return model.x[i,j] <= model.FlowCap[i,j]
model.flowcap = Constraint(model.Arcs,rule=flow_cap)

for (i,j) in model.Arcs:
  print(model.flowcap[i,j].expr)

x[1,2] + x[5,2]  ==  x[2,5]
x[1,3] + x[5,3] + x[6,3]  ==  x[3,4] + x[3,5] + x[3,6]
x[1,4] + x[3,4] + x[6,4]  ==  x[4,6] + x[4,10]
x[2,5] + x[3,5] + x[9,5]  ==  x[5,2] + x[5,3] + x[5,7] + x[5,9]
x[3,6] + x[4,6] + x[7,6] + x[8,6]  ==  x[6,3] + x[6,4] + x[6,7] + x[6,8] + x[6,10]
x[5,7] + x[6,7] + x[8,7] + x[9,7]  ==  x[7,6] + x[7,8] + x[7,9]
x[6,8] + x[7,8] + x[9,8]  ==  x[8,6] + x[8,7] + x[8,9] + x[8,10]
x[5,9] + x[7,9] + x[8,9]  ==  x[9,5] + x[9,7] + x[9,8] + x[9,10]
x[1,2] + x[1,3] + x[1,4]  ==  x[4,10] + x[6,10] + x[8,10] + x[9,10]
x[1,2]  <=  5
x[1,3]  <=  12
x[1,4]  <=  8
x[2,5]  <=  6
x[5,2]  <=  3
x[3,4]  <=  2
x[3,5]  <=  4
x[3,6]  <=  5
x[5,3]  <=  3
x[6,3]  <=  4
x[4,6]  <=  9
x[4,10]  <=  2
x[6,4]  <=  5
x[5,7]  <=  6
x[5,9]  <=  5
x[9,5]  <=  2
x[6,7]  <=  3
x[6,8]  <=  6
x[6,10]  <=  8
x[7,6]  <=  4
x[8,6]  <=  3
x[7,8]  <=  5
x[7,9]  <=  7
x[8,7]  <=  2
x[9,7]  <=  3
x[8,9]  <=  5
x[8,10]  <=  7
x[9,8]  <=  1
x[9,10]  <=  4


In [ ]:
opt.solve(model)

#Print results
print("Max Flow from 1 to 10 =",model.maxflow())
print("Decision Variables")
for a in model.Arcs:
    print(model.x[a],model.x[a].value)

Max Flow from 1 to 10 = 21.0
Decision Variables
x[1,2] 4.0
x[1,3] 9.0
x[1,4] 8.0
x[2,5] 4.0
x[5,2] 0.0
x[3,4] 0.0
x[3,5] 4.0
x[3,6] 5.0
x[5,3] 0.0
x[6,3] 0.0
x[4,6] 6.0
x[4,10] 2.0
x[6,4] 0.0
x[5,7] 4.0
x[5,9] 4.0
x[9,5] 0.0
x[6,7] 0.0
x[6,8] 3.0
x[6,10] 8.0
x[7,6] 0.0
x[8,6] 0.0
x[7,8] 4.0
x[7,9] 0.0
x[8,7] 0.0
x[9,7] 0.0
x[8,9] 0.0
x[8,10] 7.0
x[9,8] 0.0
x[9,10] 4.0


### Exercise: Implement the max flow as an abstract model

## Q2: Shortest Path - min cost

### Part a: Formulate the problem

**Sets** \\
$N$: set of nodes \\
$K$: set of nodes without source and sink \\
$A$: set of arcs $(ij)$ \\

**Parameters** \\
$c_{ij}$: cost for using arc $ij$

**Decision Variable** \\
$x_{ij}$: 1 if arc $(ij)$ is being used, 0 o.w. - binary

**Model** \\
$ min \ \sum_{(ij) \in A} c_{ij}*x_{ij}$ \\
$ s.t.$
$ \ \sum_{j \in N} x_{ij} = \sum_{j \in N} x_{ji} \quad \forall i \in K $ \\
$ \quad \quad \sum_{j \in N} x_{1j} = 1 $ \\
$ \quad \quad \sum_{i \in N} x_{i10} = 1 $ \\
$ \quad \quad x_{ij} \in \{0,1\} \quad \forall (ij) \in A $


In [ ]:
model2=ConcreteModel()

model2.Nodes=Set(initialize=range(1,11))  
model2.first=1
model2.last=10

model2.Arcs=Set(within=model2.Nodes*model2.Nodes, 
                initialize=[(1,2),(1,6),(1,7),(1,8),
                            (2,1),(6,1),(7,1),(8,1),
                            (2,4),(2,5),(2,7),(2,8),
                            (4,2),(5,2),(7,2),(8,2),
                            (3,6),(3,9),(3,10),
                            (6,3),(9,3),(10,3),
                            (4,5),(4,9),
                            (5,4),(9,4),
                            (5,7),
                            (7,5),
                            (6,8),(6,9),(6,10),
                            (8,6),(9,6),(10,6),
                            (8,9),
                            (9,8)])

model2.Arcs.pprint()

#Add parameter
model2.cost=Param(model2.Arcs,
                  initialize={(1,2):9,(1,6):5,(1,7):17,(1,8):15,
                              (2,1):9,(6,1):5,(7,1):17,(8,1):15,
                              (2,4):4,(2,5):14,(2,7):7,(2,8):6,
                              (4,2):4,(5,2):14,(7,2):7,(8,2):6,
                              (3,6):7,(3,9):2,(3,10):10,
                              (6,3):7,(9,3):2,(10,3):10,
                              (4,5):8,(4,9):11,
                              (5,4):8,(9,4):11,
                              (5,7):4,
                              (7,5):4,
                              (6,8):8,(6,9):12,(6,10):4,
                              (8,6):8,(9,6):12,(10,6):4,
                              (8,9):3,
                              (9,8):3})

#Add dec variables
#arc being used or not - decision variable xij
model2.x=Var(model2.Arcs,domain=Boolean)

#Adding objective function
def min_path(model2):
    return sum(model2.cost[a]*model2.x[a] for a in model2.Arcs)
model2.shortest=Objective(rule=min_path, sense=minimize)

print("\nObjective Function")
print(model2.shortest.expr)

#Adding constraints
def flow_rule(model2, n):
    if(n == model.first): #origin constraint
        return sum(model2.x[i,j] for (i,j) in model2.Arcs if i==n) == 1
    elif(n == model.last):  #destination constraint
        return sum(model2.x[i,j] for (i,j) in model2.Arcs if j==n) == 1
    else: #transhipment constraint
        inFlow  = sum(model2.x[i,j] for (i,j) in model2.Arcs if j == n)
        outFlow = sum(model2.x[j,i] for (j,i) in model2.Arcs if j == n)
        return inFlow == outFlow
model2.flow = Constraint(model2.Nodes, rule=flow_rule)

print("Constraints")
for n in model2.Nodes:
  print(model2.flow[n].expr)

Arcs : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain      : Size : Members
    None :     2 : Arcs_domain :   36 : {(1, 2), (1, 6), (1, 7), (1, 8), (2, 1), (6, 1), (7, 1), (8, 1), (2, 4), (2, 5), (2, 7), (2, 8), (4, 2), (5, 2), (7, 2), (8, 2), (3, 6), (3, 9), (3, 10), (6, 3), (9, 3), (10, 3), (4, 5), (4, 9), (5, 4), (9, 4), (5, 7), (7, 5), (6, 8), (6, 9), (6, 10), (8, 6), (9, 6), (10, 6), (8, 9), (9, 8)}

Objective Function
9*x[1,2] + 5*x[1,6] + 17*x[1,7] + 15*x[1,8] + 9*x[2,1] + 5*x[6,1] + 17*x[7,1] + 15*x[8,1] + 4*x[2,4] + 14*x[2,5] + 7*x[2,7] + 6*x[2,8] + 4*x[4,2] + 14*x[5,2] + 7*x[7,2] + 6*x[8,2] + 7*x[3,6] + 2*x[3,9] + 10*x[3,10] + 7*x[6,3] + 2*x[9,3] + 10*x[10,3] + 8*x[4,5] + 11*x[4,9] + 8*x[5,4] + 11*x[9,4] + 4*x[5,7] + 4*x[7,5] + 8*x[6,8] + 12*x[6,9] + 4*x[6,10] + 8*x[8,6] + 12*x[9,6] + 4*x[10,6] + 3*x[8,9] + 3*x[9,8]
Constraints
x[1,2] + x[1,6] + x[1,7] + x[1,8]  ==  1
x[1,2] + x[4,2] + x[5,2] + x[7,2] + x[8,2]  ==  x[2,1] + x[2,4] + x[2,5] + x[2,7] + x[2,8]


### Part b: Solve the model

In [ ]:
opt.solve(model2)

#Print results
print("Lowest cost from 1 to 10 =",model2.shortest())
print("Decision Variables")
for a in model2.Arcs:
    print(model2.x[a],model2.x[a].value)

Lowest cost from 1 to 10 = 9.0
Decision Variables
x[1,2] 0.0
x[1,6] 1.0
x[1,7] 0.0
x[1,8] 0.0
x[2,1] 0.0
x[6,1] 0.0
x[7,1] 0.0
x[8,1] 0.0
x[2,4] 0.0
x[2,5] 0.0
x[2,7] 0.0
x[2,8] 0.0
x[4,2] 0.0
x[5,2] 0.0
x[7,2] 0.0
x[8,2] 0.0
x[3,6] 0.0
x[3,9] 0.0
x[3,10] 0.0
x[6,3] 0.0
x[9,3] 0.0
x[10,3] 0.0
x[4,5] 0.0
x[4,9] 0.0
x[5,4] 0.0
x[9,4] 0.0
x[5,7] 0.0
x[7,5] 0.0
x[6,8] 0.0
x[6,9] 0.0
x[6,10] 1.0
x[8,6] 0.0
x[9,6] 0.0
x[10,6] 0.0
x[8,9] 0.0
x[9,8] 0.0


### Part c: Find the lowest cost from 3 to all 9 remaining nodes.

This is one approach. There are other possible ways to do it!

In [ ]:
for destination in range(1,11):
  if(destination != 3):
    model2=ConcreteModel()

    model2.Nodes=Set(initialize=range(1,11))  
    model2.first=Param(initialize = 3)
    model2.last=Param(initialize = destination)

    model2.Arcs=Set(within=model2.Nodes*model2.Nodes, 
                initialize=[(1,2),(1,6),(1,7),(1,8),
                            (2,1),(6,1),(7,1),(8,1),
                            (2,4),(2,5),(2,7),(2,8),
                            (4,2),(5,2),(7,2),(8,2),
                            (3,6),(3,9),(3,10),
                            (6,3),(9,3),(10,3),
                            (4,5),(4,9),
                            (5,4),(9,4),
                            (5,7),
                            (7,5),
                            (6,8),(6,9),(6,10),
                            (8,6),(9,6),(10,6),
                            (8,9),
                            (9,8)])

    #Add parameter
    model2.cost=Param(model2.Arcs,
                  initialize={(1,2):9,(1,6):5,(1,7):17,(1,8):15,
                              (2,1):9,(6,1):5,(7,1):17,(8,1):15,
                              (2,4):4,(2,5):14,(2,7):7,(2,8):6,
                              (4,2):4,(5,2):14,(7,2):7,(8,2):6,
                              (3,6):7,(3,9):2,(3,10):10,
                              (6,3):7,(9,3):2,(10,3):10,
                              (4,5):8,(4,9):11,
                              (5,4):8,(9,4):11,
                              (5,7):4,
                              (7,5):4,
                              (6,8):8,(6,9):12,(6,10):4,
                              (8,6):8,(9,6):12,(10,6):4,
                              (8,9):3,
                              (9,8):3})

    #Add dec variables
    model2.x=Var(model2.Arcs,domain=Boolean)

    #Adding objective function
    def min_path(model2):
      return sum(model2.cost[a]*model2.x[a] for a in model2.Arcs)
    model2.shortest=Objective(rule=min_path, sense=minimize)

    #Adding constraints
    def flow_rule(model2, n):
       if(n == model2.first): #origin constraint
          return sum(model2.x[i,j] for (i,j) in model2.Arcs if i==n) == 1
       elif(n == model2.last):  #destination constraint
          return sum(model2.x[i,j] for (i,j) in model2.Arcs if j==n) == 1
       else: #transhipment constraint
          inFlow  = sum(model2.x[i,j] for (i,j) in model2.Arcs if j == n)
          outFlow = sum(model2.x[j,i] for (j,i) in model2.Arcs if j == n)
          return inFlow == outFlow
    model2.flow = Constraint(model2.Nodes, rule=flow_rule)
    
    opt.solve(model2)

    #Print results
    print("Lowest cost from 3 to",destination,"=",model2.shortest())
    for (i,j) in model2.Arcs:
      if model2.x[i,j].value != 0:
        print(model2.x[i,j],model2.x[i,j].value)

Lowest cost from 3 to 1 = 12.0
x[6,1] 1.0
x[3,6] 1.0
Lowest cost from 3 to 2 = 11.0
x[8,2] 1.0
x[3,9] 1.0
x[9,8] 1.0
Lowest cost from 3 to 4 = 12.0
x[2,4] 1.0
x[4,2] 1.0
x[3,9] 1.0
x[9,3] 1.0
Lowest cost from 3 to 5 = 12.0
x[3,9] 1.0
x[9,3] 1.0
x[5,7] 1.0
x[7,5] 1.0
Lowest cost from 3 to 6 = 7.0
x[3,6] 1.0
Lowest cost from 3 to 7 = 12.0
x[3,9] 1.0
x[9,3] 1.0
x[5,7] 1.0
x[7,5] 1.0
Lowest cost from 3 to 8 = 5.0
x[3,9] 1.0
x[9,8] 1.0
Lowest cost from 3 to 9 = 2.0
x[3,9] 1.0
Lowest cost from 3 to 10 = 10.0
x[3,10] 1.0


Note that some path are non contiguous. For example 3 to 7, it goes from 3 to 9 then back to 3 and from 5 to 7 and then back to 5. We can add a new contraitn to make sure each arc is only used in one direction. In other words if \\
$x_{39} = 1$ then $x_{93} = 0$ \\
One way to do it is by adding a constraint \\
$x_{39} + x_{93} = 1$ \\
But the problem is this constraint would force arc (3,9) or (9,3) to be in the optimal solution because one of them need to be one for the sum to be one. So the constraint should be modified such that \\
$x_{39} + x_{93} <= 1$ \\
Now we allow both to be zero and one of them to be 1 in the optimal solution. Let's add this to the model.

In [ ]:
for destination in range(1,11):
  if(destination != 3):
    model2=ConcreteModel()

    model2.Nodes=Set(initialize=range(1,11))  
    model2.first=Param(initialize = 3)
    model2.last=Param(initialize = destination)

    model2.Arcs=Set(within=model2.Nodes*model2.Nodes, 
                initialize=[(1,2),(1,6),(1,7),(1,8),
                            (2,1),(6,1),(7,1),(8,1),
                            (2,4),(2,5),(2,7),(2,8),
                            (4,2),(5,2),(7,2),(8,2),
                            (3,6),(3,9),(3,10),
                            (6,3),(9,3),(10,3),
                            (4,5),(4,9),
                            (5,4),(9,4),
                            (5,7),
                            (7,5),
                            (6,8),(6,9),(6,10),
                            (8,6),(9,6),(10,6),
                            (8,9),
                            (9,8)])

    #Add parameter
    model2.cost=Param(model2.Arcs,
                  initialize={(1,2):9,(1,6):5,(1,7):17,(1,8):15,
                              (2,1):9,(6,1):5,(7,1):17,(8,1):15,
                              (2,4):4,(2,5):14,(2,7):7,(2,8):6,
                              (4,2):4,(5,2):14,(7,2):7,(8,2):6,
                              (3,6):7,(3,9):2,(3,10):10,
                              (6,3):7,(9,3):2,(10,3):10,
                              (4,5):8,(4,9):11,
                              (5,4):8,(9,4):11,
                              (5,7):4,
                              (7,5):4,
                              (6,8):8,(6,9):12,(6,10):4,
                              (8,6):8,(9,6):12,(10,6):4,
                              (8,9):3,
                              (9,8):3})

    #Add dec variables
    model2.x=Var(model2.Arcs,domain=Boolean)

    #Adding objective function
    def min_path(model2):
      return sum(model2.cost[a]*model2.x[a] for a in model2.Arcs)
    model2.shortest=Objective(rule=min_path, sense=minimize)

    #Adding constraints
    def flow_rule(model2, n):
       if(n == model2.first): #origin constraint
          return sum(model2.x[i,j] for (i,j) in model2.Arcs if i==n) == 1
       elif(n == model2.last):  #destination constraint
          return sum(model2.x[i,j] for (i,j) in model2.Arcs if j==n) == 1
       else: #transhipment constraint
          inFlow  = sum(model2.x[i,j] for (i,j) in model2.Arcs if j == n)
          outFlow = sum(model2.x[j,i] for (j,i) in model2.Arcs if j == n)
          return inFlow == outFlow
    model2.flow = Constraint(model2.Nodes, rule=flow_rule)

    ## added constraint to fix issue from previous solution
    def no_back(model2,i,j):
       return model2.x[i,j] + model2.x[j,i] <= 1
    model2.noback = Constraint(model2.Arcs, rule=no_back)
    
    opt.solve(model2)

    #Print results
    print("Lowest cost from 3 to",destination,"=",model2.shortest())
    for (i,j) in model2.Arcs:
      if model2.x[i,j].value != 0:
        print(model2.x[i,j],model2.x[i,j].value)

Lowest cost from 3 to 1 = 12.0
x[6,1] 1.0
x[3,6] 1.0
Lowest cost from 3 to 2 = 11.0
x[8,2] 1.0
x[3,9] 1.0
x[9,8] 1.0
Lowest cost from 3 to 4 = 13.0
x[3,9] 1.0
x[9,4] 1.0
Lowest cost from 3 to 5 = 21.0
x[3,9] 1.0
x[4,5] 1.0
x[9,4] 1.0
Lowest cost from 3 to 6 = 7.0
x[3,6] 1.0
Lowest cost from 3 to 7 = 18.0
x[2,7] 1.0
x[8,2] 1.0
x[3,9] 1.0
x[9,8] 1.0
Lowest cost from 3 to 8 = 5.0
x[3,9] 1.0
x[9,8] 1.0
Lowest cost from 3 to 9 = 2.0
x[3,9] 1.0
Lowest cost from 3 to 10 = 10.0
x[3,10] 1.0


## Q4: Environmental Strategy

**Sets** \\
$P$: set of projects \\
$I$: set of impacts \\
$B$: set of business line \\

**Parameters** \\
$c_{p}$: price per project $p$ \\
$PIU_{p,i}$: positive impact unit on $i$ for project $p$ \\
$DImax_{b,i}$: maximum desired impact $i$ for business $b$  \\
$DImin_{b,i}$: minimum desired impact $i$ for business $b$  \\
$Bmin_{b}$: minimum investment in each business $i$ \\
$PImax_{p}$: maximum number of feasible projects for $p$ \\

**Decision Variable** \\
$x_{pb}$: how many project type $p$ to invest on for industry $k$

**Model** \\
$ min \ \sum_{i} \sum_{p} c_{p}*x_{pb}$ \\
$ s.t.$
$ \ \sum_{b} x_{pb} \leq PImax_{p} \quad \forall p $ \\
$ \quad \quad \sum_{p} x_{pb} \geq Bmin_{b} \quad \forall b $ \\
$ \quad \quad \sum_{p} PIU_{p,i}*x_{p,b} \geq \sum_{p} x_{p,b} * DImin_{b,i} \quad \forall i,b $ \\
$ \quad \quad \sum_{p} PIU_{p,i}*x_{p,b} \leq \sum_{p} x_{p,b} * DImax_{b,i} \quad \forall i,b $ \\
$ \quad \quad x_{pb} \geq 0 \quad \forall p,b$




In [ ]:
import numpy as np

#Entering data
PIU = np.array([[8, 5, 6, 5],[6, 4, 12, 4],[11, 12, 7, 7],[7, 10, 5, 9]])
DImin = np.array([[6, 8, 7, 5],[7, 5, 5, 4],[4, 6, 4, 3]])
DImax = np.array([[10**10, 10**10, 10**10, 8],[10**10, 10**10, 10**10, 6],[8, 10**10, 10**10, 6]])
c = np.array([2.1,1.3,2.7,1.7])
Bmin = np.array([140,700,600])
PImax = np.array([700,1100,600,500])

#Creating model
model4 = ConcreteModel()

model4.I = Set(initialize=range(4)) #["Climate","Water","Ecosystem","Human_health"]
model4.P = Set(initialize=range(4)) #["Electrification","Eco_Restauration","Water_Retro","Bulding_Retro"]
model4.B = Set(initialize=range(3)) #["A","B","C"]

model4.x = Var(model4.P,model4.B,domain = NonNegativeReals)

#add objective
model4.cost = Objective(sense=minimize, 
                       expr=sum(sum(c[p]*model4.x[p,b] for b in model4.B) for p in model4.P))

#add constraints
def max_project(model4,p):
  return sum(model4.x[p,b] for b in model4.B) <= PImax[p]
model4.maxproject = Constraint(model4.P,rule=max_project)

def min_industry(model4,b):
  return sum(model4.x[p,b] for p in model4.P) >= Bmin[b]
model4.minindustry = Constraint(model4.B,rule=min_industry)

def max_impact(model4,b,i):
  return sum(PIU[p,i]*model4.x[p,b] for p in model4.P) <= sum(model4.x[p,b] for p in model4.P)*DImax[b,i]
model4.maximpact = Constraint(model4.B,model4.I,rule=max_impact)

def min_impact(model4,b,i):
  return sum(PIU[p,i]*model4.x[p,b] for p in model4.P) >= sum(model4.x[p,b] for p in model4.P)*DImin[b,i]
model4.minimpact = Constraint(model4.B,model4.I,rule=min_impact)

opt.solve(model4)
print("Total cost =",model4.cost())

for b in model4.B:
  for p in model4.P:
    print(model4.x[p,b],model4.x[p,b].value)

Total cost = 2185.3333332
x[0,0] 0.0
x[1,0] 46.666667
x[2,0] 0.0
x[3,0] 93.333333
x[0,1] 0.0
x[1,1] 560.0
x[2,1] 140.0
x[3,1] 0.0
x[0,2] 0.0
x[1,2] 400.0
x[2,2] 0.0
x[3,2] 200.0


In [ ]:
#Printing model to check objective function and constraints
print(model4.cost.expr)

for p in model4.P:
  print(model4.maxproject[p].expr)

for b in model4.B:
  print(model4.minindustry[b].expr)

for b in model4.B:
  for i in model4.I:
    print(model4.maximpact[b,i].expr)

for b in model4.B:
  for i in model4.I:
    print(model4.minimpact[b,i].expr)


2.1*x[0,0] + 2.1*x[0,1] + 2.1*x[0,2] + 1.3*x[1,0] + 1.3*x[1,1] + 1.3*x[1,2] + 2.7*x[2,0] + 2.7*x[2,1] + 2.7*x[2,2] + 1.7*x[3,0] + 1.7*x[3,1] + 1.7*x[3,2]
x[0,0] + x[0,1] + x[0,2]  <=  700
x[1,0] + x[1,1] + x[1,2]  <=  1100
x[2,0] + x[2,1] + x[2,2]  <=  600
x[3,0] + x[3,1] + x[3,2]  <=  500
140  <=  x[0,0] + x[1,0] + x[2,0] + x[3,0]
700  <=  x[0,1] + x[1,1] + x[2,1] + x[3,1]
600  <=  x[0,2] + x[1,2] + x[2,2] + x[3,2]
8*x[0,0] + 6*x[1,0] + 11*x[2,0] + 7*x[3,0]  <=  (x[0,0] + x[1,0] + x[2,0] + x[3,0])*inf
5*x[0,0] + 4*x[1,0] + 12*x[2,0] + 10*x[3,0]  <=  (x[0,0] + x[1,0] + x[2,0] + x[3,0])*inf
6*x[0,0] + 12*x[1,0] + 7*x[2,0] + 5*x[3,0]  <=  (x[0,0] + x[1,0] + x[2,0] + x[3,0])*inf
5*x[0,0] + 4*x[1,0] + 7*x[2,0] + 9*x[3,0]  <=  (x[0,0] + x[1,0] + x[2,0] + x[3,0])*8.0
8*x[0,1] + 6*x[1,1] + 11*x[2,1] + 7*x[3,1]  <=  (x[0,1] + x[1,1] + x[2,1] + x[3,1])*inf
5*x[0,1] + 4*x[1,1] + 12*x[2,1] + 10*x[3,1]  <=  (x[0,1] + x[1,1] + x[2,1] + x[3,1])*inf
6*x[0,1] + 12*x[1,1] + 7*x[2,1] + 5*x[3,1]  <=  (x[